In [1]:
using JuMP, Ipopt, SparseArrays, LinearAlgebra, NamedArrays, DelimitedFiles, DataFrames, CSV, Statistics

# Include necessary functions
include("Functions/Ybus.jl");         # Function to calculate Ybus matrix (G,B)
include("Functions/Conventional.jl");   # Function to define constraints (polar)
include("Functions/Cost.jl");          # Function to define cost (polar)
include("Functions/Misc.jl");                # Function to define start point,map input,print output
include("Functions/Parse_gld_1day.jl");      # Function to parse GridLabD system data
include("Functions/Fixed_point.jl");     # Function to implement fixed-point method (polar)
include("Functions/BFS.jl");             # Function to implement BFS method (polar)
include("Functions/Dist3Flow.jl");       # Function to implement Dist3Flow method

# Load test case
global case_name = "case123_glm" 
# global case_name = "GC-12.47-1" 
# global case_name = "R1-12.47-3/base"  
# global case_name = "R2-12.47-2"
data_file = open(string("data/",case_name,"/data.xml")); data = readlines(data_file)
z_file = open(string("data/",case_name,"/z_dump.xml")); z_dump = readlines(z_file)
global PV_en = 0;   # 0-exclude, 1-include ZIP loads/PV inverters
if PV_en == 1
    house_sch = CSV.read(string("House_",case_name,"_profile.csv"),header=false,type=Float64,DataFrame)
    global house_sch = convert(Matrix,house_sch);
    PV_sch = CSV.read(string("PV_",case_name,"_profile.csv"),header=false,type=Float64,DataFrame)
    global PV_sch = convert(Matrix,PV_sch);
end
# global VUF_init = readdlm(string("VUF_",case_name,"_init.csv"), ',')/100
# global QgY_init = readdlm(string("QgY0_",case_name,"_base.csv"), ',');
# global QgY_init = readdlm("QgY0_R1-12.47-1_freq60_delay60.csv", ',');

## Other inputs
global count_sch = 1;  # Time instant to run OPF
# global crit_node = ["n632"]
# global crit_node = ["R1-12-47-1_node_359"]
# global crit_node = ["R1-12-47-3_node_1"]
global crit_node = "all"
global PV_rat = 0.035;
global Q_pen = 0*1e-5; global ΔV_pen = 0*1e-2;
global obj_op = 1       
        
## Parse system data
parse_gld(crit_node); global M = 1e3

## Create Y_bus and connection matrices
Ybus(); conn_bgl();

# Initialize values
start_point(4); global Vm0 = s0[1:nb_nph]; global θ0 = s0[nb_nph+1:2*nb_nph];
global Vd0 = Vm0.*cos.(θ0); global Vq0 = Vm0.*sin.(θ0); QgY0 = zeros(ngy_nph-3);
stat = "LOCALLY_SOLVED";
round.(bus[:,9:14],RoundNearest, digits=4)

par_det = 8×1 Named Adjoint{Int64,Array{Int64,2}}
Par ╲ Val │ :Value
──────────┼───────
nb        │    134
ngy       │      1
nly       │     82
nld       │      3
nb_nph    │    286
ngy_nph   │      3
nly_nph   │    144
nld_nph   │      9
-------------------


134×6 Array{Float64,2}:
 0.0      0.0     0.0        0.0    1.0332  119.592
 1.0162  -1.4398  1.0386  -120.728  1.0252  119.203
 1.0083  -1.8789  1.0365  -120.962  1.0195  118.92 
 1.0067  -1.5019  0.0        0.0    0.0       0.0  
 1.0084  -1.4685  0.0        0.0    0.0       0.0  
 0.0      0.0     0.0        0.0    1.0182  118.894
 0.9995  -2.3286  1.0325  -121.201  1.0117  118.857
 0.999   -2.3838  1.0326  -121.199  1.0105  118.836
 0.9986  -2.4394  1.033   -121.183  1.0094  118.82 
 0.9979  -2.4981  1.0335  -121.174  1.0085  118.821
 0.9977  -2.5252  0.0        0.0    1.0017  118.811
 0.9979  -2.4981  0.0        0.0    1.0022  118.821
 0.9973  -2.5437  0.0        0.0    1.0016  118.82 
 ⋮                                            ⋮    
 1.0353  -3.8265  1.0303  -122.228  1.0333  117.585
 1.0355  -3.8296  1.0302  -122.215  1.0328  117.567
 0.0      0.0     0.0        0.0    1.0318  117.597
 0.0      0.0     0.0        0.0    1.0301  117.566
 0.0      0.0     0.0        0.0    1.02

In [156]:
## Define active and reactive power of inverters
    Ygen = zeros(nb_nph)*(0+im*0)
    if PV_en == 1
        Pg_idx = [2 6 10];  Qg_idx = [4 8 12];
        global PgY = zeros(ngy_inv); SgY = yinv_ratedS/baseMVA; num_count = 1;
        for i=1:ngy_inv
            j = Int(ygen1[i,1]); count1 = 1
            for k = 3:2:7
                if bus[j,k] != 0
                    PgY[num_count] = ygen1[i,Pg_idx[count1]]/baseMVA
                    if QgY0[num_count] > sqrt(SgY[num_count]^2 - PgY[num_count]^2)
                        QgY0[num_count] = sqrt(SgY[num_count]^2 - PgY[num_count]^2)
                    elseif QgY0[num_count] < -sqrt(SgY[num_count]^2 - PgY[num_count]^2)
                        QgY0[num_count] = -sqrt(SgY[num_count]^2 - PgY[num_count]^2)
                    end
                    num_count = num_count + 1;
                end
                count1 = count1 + 1;
            end
        end
        Ygen = Cbg*(PgY + im*QgY0)
    end

    ## Generate load matrix
    ZIP_load()

Vll0 = zeros(nb_nph); pf_err = 1; pf_iter = 0; Ibr = zeros(nbr_nph)*(0+im*0);
    YloadP = zeros(nly_nph); YloadQ = zeros(nly_nph); Vm1 = zeros(nb_nph);
    DloadP = zeros(nld_nph); DloadQ = zeros(nld_nph); θ1 = zeros(nb_nph);
    I_ll = zeros(nld_nph)*(0+im*0); P_d2y = zeros(nldy_nph); Q_d2y = zeros(nldy_nph);

In [161]:
## Initialize voltage
        for i=1:nb_nph
            k = Int(Tl[i])
            Vll0[i] =  sqrt(Vm0[i]^2 + Vm0[k]^2 - 2*Vm0[i]*Vm0[k]*cos(θ0[i]-θ0[k]) )
        end

        ## Calculate wye-connected loads
        for i=1:nl_y
            dum1 = (Sy[i,3] + im*Sy[i,4])/(1*exp(im*Sy[i,8]))*(Vm0[Int(Sy[i,7])]*exp(im*θ0[Int(Sy[i,7])]))
            YloadP[i] = Sy[i,1]+real(dum1)+Sy[i,5]*Vm0[Int(Sy[i,7])]^2
            YloadQ[i] = Sy[i,2]+imag(dum1)+Sy[i,6]*Vm0[Int(Sy[i,7])]^2

        end
        for i=nl_y+1:nly_nph
            YloadP[i] = Sy[i,1]+Sy[i,3]*Vm0[Int(Sy[i,7])]+Sy[i,5]*Vm0[Int(Sy[i,7])]^2
            YloadQ[i] = Sy[i,2]+Sy[i,4]*Vm0[Int(Sy[i,7])]+Sy[i,6]*Vm0[Int(Sy[i,7])]^2
        end
        Ycap = Vm0.*exp.(im*θ0).*conj.(im*Sh_cap.*Vm0.*exp.(im*θ0)).*Vm0.^2
        Yload = CblY*(YloadP + im*YloadQ) + Ycap;

        ## Calculate delta-connected loads
        for i=1:nld_nph
            j = Int(Tl[Int(Sd[i,7])]);
            dum1 = Vm0[Int(Sd[i,7])]*exp(im*θ0[Int(Sd[i,7])])-Vm0[j]*exp(im*θ0[j])
            dum2 = (Sd[i,3] + im*Sd[i,4])/(sqrt(3)*exp(im*Sd[i,8]))*dum1
            DloadP[i] = Sd[i,1]+real(dum2)+Sd[i,5]*Vll0[Int(Sd[i,7])]^2/3
            DloadQ[i] = Sd[i,2]+imag(dum2)+Sd[i,6]*Vll0[Int(Sd[i,7])]^2/3
        end
        num_count= 1;
        for i=1:nld
            j = Int(dload[i,1])
            for k=Int.(sum(bus_count[1:j-1])+1:sum(bus_count[1:j]))
                kk = Int(Tl[k])
                if k != kk
                    I_ll[num_count]= conj((DloadP[num_count]+im*DloadQ[num_count])/(Vm0[k]*exp(im*θ0[k])-Vm0[kk]*exp(im*θ0[kk])));
                    num_count= num_count+ 1
                end
            end
        end
        Ip = Tl2p*I_ll;
        num_count= 1
        for i=1:nld
            j = Int(dload[i,1])
            for k=Int.(sum(bus_count[1:j-1])+1:sum(bus_count[1:j]))
                dum1 = Vm0[k]*exp(im*θ0[k])*conj(Ip[num_count])
                P_d2y[num_count]= real(dum1); Q_d2y[num_count]= imag(dum1)
                num_count= num_count+ 1
            end
        end
        Dload = CblD*(P_d2y + im*Q_d2y); Ibus = zeros(nb_nph)*(0+im*0);
        V0 = Vm0.*exp.(im*θ0); S_inj = Dload+Yload-Ygen; count3 = 1;
        I_inj = conj(S_inj./V0)

286-element Array{Complex{Float64},1}:
                   0.0 + 0.0im                
                   0.0 - 0.0im                
                  -0.0 + 0.0im                
                   0.0 + 0.0im                
                   0.0 - 0.0im                
                  -0.0 + 0.0im                
                   0.0 + 0.0im                
                   0.0 - 0.0im                
                   0.0 - 0.0im                
                   0.0 + 0.0im                
                   0.0 - 0.0im                
                  -0.0 + 0.0im                
                   0.0 + 0.0im                
                       ⋮                      
                  -0.0 + 0.0im                
 -0.012581878286256778 + 1.0595209233303398im 
 -0.006765449538361715 + 0.5443738009837381im 
 -0.012774185527721284 + 1.0910697376937961im 
 -0.011951931201792835 + 1.093583531918059im  
   -0.9041240897923306 - 0.5362509829748676im 
   -0.906022969758767

In [160]:
[bus_ID[Int.(tf_branch[120:123,1:2])] tf_branch[120:123,:]]
abs.(I_inj[20:30])

11-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [162]:
## Backward sweep

        for i=1:nbr
            j = Int(line_prove[nbr-i+1,2]); jj = Int(sum(bus_count[1:j-1])+1):Int(sum(bus_count[1:j]))
            k = Int(line_prove[nbr-i+1,1]); kk = Int(sum(bus_count[1:k-1])+1):Int(sum(bus_count[1:k]))
            nz = findmin([length(jj) length(kk)])[1]
            jjjj = zeros(nz); kkkk = zeros(nz); num_count=1
            for jjj=jj
                for kkk=kk
                    if bus_ϕ[jjj] == bus_ϕ[kkk]
                        jjjj[num_count] = jjj; kkkk[num_count] = kkk;
                        num_count=num_count+1
                    end
                end
            end
            jj = Int.(jjjj); kk = Int.(kkkk)
            if tf_branch[Int(line_prove[nbr-i+1,4]),3] == 8
                if bus_ph[k] == "AN"
                    vr_tap = 1 + 0.00625*tf_branch[vr_idx[count3],6]
                elseif bus_ph[k] == "BN"
                    vr_tap = 1 + 0.00625*tf_branch[vr_idx[count3],7]
                elseif bus_ph[k] == "CN"
                    vr_tap = 1 + 0.00625*tf_branch[vr_idx[count3],8]
                elseif bus_ph[k] == "ABN" || bus_ph[k] == "BAN"
                    vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count3],6:7])
                elseif bus_ph[k] == "BCN" || bus_ph[k] == "CBN"
                    vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count3],7:8])
                elseif bus_ph[k] == "ACN" || bus_ph[k] == "CAN"
                    vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count3],6:2:8])
                else
                    vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count3],6:8])
                end
                Ibus[kk] = vr_tap*(Ibus[kk] + I_inj[kk]);
                count3 = count3+1;
            else
                Ibus[kk] = Ibus[kk] + I_inj[kk];
            end
            Ibus[jj] = Ibus[kk] + Ibus[jj];
end
abs.(Ibus[20:30])

11-element Array{Float64,1}:
 3.2527029456322665
 0.0               
 2.1597788337745083
 1.073312969700693 
 1.0408307885803267
 1.1202704121250981
 1.0863671355807158
 1.073312969700693 
 0.0               
 1.056921571585906 
 1.059577973275181 

In [163]:
sum(Ibus)

139.71020157643923 - 15.682202867993796im

In [148]:
i=25
j = Int(line_prove[nbr-i+1,2]); jj = Int(sum(bus_count[1:j-1])+1):Int(sum(bus_count[1:j]))
            k = Int(line_prove[nbr-i+1,1]); kk = Int(sum(bus_count[1:k-1])+1):Int(sum(bus_count[1:k]))
            nz = findmin([length(jj) length(kk)])[1]
            jjjj = zeros(nz); kkkk = zeros(nz); num_count=1
            for jjj=jj
                for kkk=kk
                    if bus_ϕ[jjj] == bus_ϕ[kkk]
                        jjjj[num_count] = jjj; kkkk[num_count] = kkk;
                        num_count=num_count+1
                    end
                end
            end
            jj = Int.(jjjj); kk = Int.(kkkk)

Ibus[kk]

3-element Array{Complex{Float64},1}:
   0.921911289685281 - 0.5231873022436246im
                 0.0 + 0.0im               
 -0.0384271189732162 + 1.0728477987870568im

In [144]:
I_inj[kk]

3-element Array{Complex{Float64},1}:
 0.959999151696 - 0.48000245798400004im
           -0.0 + 0.0im                
            0.0 + 0.0im                

In [151]:
## Forward sweep
        Vm1[idx_ref] = Vm0[idx_ref]; θ1[idx_ref] = θ0[idx_ref]; count1 = 1;  count2=1
        for i=1:nbr
            j = Int(line_prove[i,2]); jj = Int(sum(bus_count[1:j-1])+1):Int(sum(bus_count[1:j]))
            k = Int(line_prove[i,1]); kk = Int(sum(bus_count[1:k-1])+1):Int(sum(bus_count[1:k]))
            nz = findmin([length(jj) length(kk)])[1]
            jjjj = zeros(nz); kkkk = zeros(nz); num_count=1
            for jjj=jj
                for kkk=kk
                    if bus_ϕ[jjj] == bus_ϕ[kkk]
                        jjjj[num_count] = jjj; kkkk[num_count] = kkk;
                        num_count=num_count+1
                    end
                end
            end
            jj = Int.(jjjj); kk = Int.(kkkk); Zbr = -pinv(Y[jj,kk]);
            if tf_branch[Int(line_prove[i,4]),3] == 8
                if bus_ph[k] == "AN"
                    vr_tap = 1 + 0.00625*tf_branch[vr_idx[count2],6]
                elseif bus_ph[k] == "BN"
                    vr_tap = 1 + 0.00625*tf_branch[vr_idx[count2],7]
                elseif bus_ph[k] == "CN"
                    vr_tap = 1 + 0.00625*tf_branch[vr_idx[count2],8]
                elseif bus_ph[k] == "ABN" || bus_ph[k] == "BAN"
                    vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count2],6:7])
                elseif bus_ph[k] == "BCN" || bus_ph[k] == "CBN"
                    vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count2],7:8])
                elseif bus_ph[k] == "ACN" || bus_ph[k] == "CAN"
                    vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count2],6:2:8])
                else
                    vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count2],6:8])
                end
                Vm_kk = vr_tap*Vm1[jj].*exp.(im*θ1[jj]) - Zbr*Ibus[kk]
                count2 = count2+1;
            else
                Vm_kk = Vm1[jj].*exp.(im*θ1[jj]) - Zbr*Ibus[kk]
            end
            # Vm_jj = Vm1[jj].*exp.(im*θ1[jj]); ii = Int(line_prove[i,4])
            # if bus_ph[k] == "AN" || bus_ph[k] == "AS" || bus_ph[k] == "A"
            #     Vm_kk = A_mat[1,1,ii]*Vm_jj - B_mat[1,1,ii]*Ibr[count1:count1+length(kk)-1]
            # elseif bus_ph[k] == "BN" || bus_ph[k] == "BS" || bus_ph[k] == "B"
            #     Vm_kk = A_mat[2,2,ii]*Vm_jj - B_mat[2,2,ii]*Ibr[count1:count1+length(kk)-1]
            # elseif bus_ph[k] == "CN" || bus_ph[k] == "CS" || bus_ph[k] == "C"
            #     Vm_kk = A_mat[3,3,ii]*Vm_jj - B_mat[3,3,ii]*Ibr[count1:count1+length(kk)-1]
            # elseif bus_ph[k] == "ABN" || bus_ph[k] == "ABD" || bus_ph[k] == "BAN" || bus_ph[k] == "BAD" || bus_ph[k] == "BA" || bus_ph[k] == "AB"
            #     Vm_kk = A_mat[1:2,1:2,ii]*Vm_jj - B_mat[1:2,1:2,ii]*Ibr[count1:count1+length(kk)-1]
            # elseif bus_ph[k] == "BCN" || bus_ph[k] == "BCD" || bus_ph[k] == "CBN"  || bus_ph[k] == "CBD" || bus_ph[k] == "CB" || bus_ph[k] == "BC"
            #     Vm_kk = A_mat[2:3,2:3,ii]*Vm_jj - B_mat[2:3,2:3,ii]*Ibr[count1:count1+length(kk)-1]
            # elseif bus_ph[k] == "ACN" || bus_ph[k] == "ACD" || bus_ph[k] == "CAN" || bus_ph[k] == "CAD" || bus_ph[k] == "CA" || bus_ph[k] == "AC"
            #     Vm_kk = A_mat[1:2:3,1:2:3,ii]*Vm_jj - B_mat[1:2:3,1:2:3,ii]*Ibr[count1:count1+length(kk)-1]
            # else
            #     Vm_kk = A_mat[:,:,ii]*Vm_jj - B_mat[:,:,ii]*Ibr[count1:count1+length(kk)-1]
            # end
            Vm1[kk] = abs.(Vm_kk); θ1[kk] = angle.(Vm_kk);
            count1 = count1+length(kk);
        end
Vm1

286-element Array{Float64,1}:
 1.0308940449047042
 1.0102031068980184
 1.0378706590132938
 1.0215328900635976
 1.0006928805680577
 1.0355359674048685
 1.0147795567461513
 1.0001294053069554
 1.0021341001018431
 1.013129762047636 
 0.990132763773248 
 1.0309256128553883
 1.0053728165472773
 ⋮                 
 1.021600489414898 
 0.9872197368740786
 0.9857470622197908
 0.9836576315736499
 0.981405171231452 
 1.0210429767379892
 1.01926231666391  
 0.963477290676147 
 0.960177364877557 
 0.9603471328665241
 0.9578928525166114
 0.9573570058421867

In [152]:
## Update solution
        V1 = Vm1.*exp.(im*θ1); pf_err = sum(abs.(V1-V0)); S_inj = V1.*conj(Y*V1);
        Vm0 = Vm1; θ0 = θ1;
        for i=1:nb_nph
            k = Int(Tl[i])
            Vll0[i] =  sqrt(Vm0[i]^2 + Vm0[k]^2 - 2*Vm0[i]*Vm0[k]*cos(θ0[i]-θ0[k]) )
        end
        pf_iter = pf_iter + 1;


In [155]:
P_inj = round.(real(S_inj),RoundNearest, digits=3)
P_inj[20:30]

11-element Array{Float64,1}:
 -225370.337
       0.0  
 -172464.141
      -0.0  
      -0.0  
  236638.758
  178972.136
       0.0  
       0.0  
       0.0  
      -0.0  